In [10]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense


data = pd.read_csv("D:/giri/downloads/archive/test.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0          im updating my blog because i feel shitty  sadness
1  i never make her separate from me because i do...  sadness
2  i left with my bouquet of red and yellow tulip...      joy
3    i was feeling a little vain when i did this one  sadness
4  i cant walk into a shop anywhere where i do no...     fear


In [11]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [12]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [13]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [14]:
one_hot_labels = keras.utils.to_categorical(labels)

In [15]:
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

In [16]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
50/50 [==============================] - 2s 21ms/step - loss: 1.5902 - accuracy: 0.3240 - val_loss: 1.5660 - val_accuracy: 0.3500
Epoch 2/10
50/50 [==============================] - 1s 17ms/step - loss: 1.3846 - accuracy: 0.5053 - val_loss: 1.5493 - val_accuracy: 0.3725
Epoch 3/10
50/50 [==============================] - 1s 18ms/step - loss: 0.9072 - accuracy: 0.6904 - val_loss: 1.5219 - val_accuracy: 0.4125
Epoch 4/10
50/50 [==============================] - 1s 21ms/step - loss: 0.4082 - accuracy: 0.9174 - val_loss: 1.6462 - val_accuracy: 0.3650
Epoch 5/10
50/50 [==============================] - 1s 20ms/step - loss: 0.1418 - accuracy: 0.9837 - val_loss: 1.6743 - val_accuracy: 0.4000
Epoch 6/10
50/50 [==============================] - 1s 17ms/step - loss: 0.0543 - accuracy: 0.9969 - val_loss: 1.7323 - val_accuracy: 0.4175
Epoch 7/10
50/50 [==============================] - 1s 16ms/step - loss: 0.0243 - accuracy: 1.0000 - val_loss: 1.7866 - val_accuracy: 0.4175
Epoch 8/10
50

In [17]:
input_text = "She didn't come today because she lost her dog yestertay!"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 [==============================] - 0s 118ms/step
['sadness']
